In [1]:
# Install libraries.

!pip install selenium
!pip install pymongo

In [122]:
# Importing required libraries.

import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

### Parcial Práctico de Big Data para el Segundo Corte.


### - Conexión al sitio web a analizar y manipulación del buscador para realizar scraping. 


In [3]:
# Base URL of the site to be analyzed.
SITE_URL = 'https://es.wikipedia.org'

In [4]:
# Chrome web driver path
# Download the driver for you S.O. here: https://chromedriver.storage.googleapis.com/index.html?path=94.0.4606.41/
DRIVER_PATH = './chromedriver.exe'

In [5]:
# Creating a new Chrome window.
browser = webdriver.Chrome(executable_path = DRIVER_PATH)

C:\Users\toshiba\AppData\Local\Temp/ipykernel_12560/911755350.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser.
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load.
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis.
    return BeautifulSoup(browser.page_source, 'html.parser')

### Extracción de los datos solicitados.

In [8]:
# Getting HTML content for news listing page

#Se tomó esta página poque era la más completa para los datos solicitados. Otras páginas web solo poseían nombre
#y periodo pero no partdio político. Así que, en cuestión de análisis, esta es la más completa. 

soup = make_request(browser, '/wiki/Anexo:Presidentes_de_Colombia')

In [345]:
# Finding the section where news are contained 

#Aquí se encuentra un problema porque, debido a la estructura de la página, todas las tablas contenidas en esta
#no poseen un id que las diferencie. Por tanto, solo encuentra la primera. 

layout = soup.find(class_ = 'wikitable')


In [356]:
#Se convierte toda la tabla en texto, usando la función getText(), ya que al intentar realizar un scraping más
#profundo, no se logra obtener todos los datos sino solo de una etiqueta pues, al usar la función find_All, esta
#convierte todo a una lista inmanipulable mediante las funciones de la libería: BeautifulSoup.

print(layout.getText())



Período
Efigie
Presidente
Ciudades

Alineación
Comentarios




25 de julio de 1810-1 de abril de 1811

(e) José Miguel Pey y Andrade (1er. periodo)
Santafé

Centralista

Origen del mando: Encargo.
Presidente de la Junta Suprema de Santafé de Bogotá tras la dimisión del Virrey Amar y Borbón.




1 de abril de 1811-19 de septiembre de 1811

Jorge Tadeo Lozano
Santafé

Federalista

Origen del mando: Elegido.
Presidente del Estado de Cundinamarca y Vicerregente de Nueva Granada.Vicepresidente: José María Domínguez del Castillo.
Familiaː Hijo del primer marqués de San Jorge
Murió fusilado bajo la Reconquista.




30 de septiembre de 1813-29 de agosto de 1815

Antonio José de Nariño y Álvarez
Santafé

Centralista

Origen del mando: Elegido tras la renuncia de Jorge Tadeo Lozano.
Presidente del Estado de Cundinamarca.
Fue el prócer que más años pasó en prisión.




23 de diciembre de 1811 -24 de diciembre de 1811



Pedro Groot y Alea
Santafé

Centralista


Presidente provisional de las Pro

In [358]:
#Getting blocks from layout

#Acá se ejempliica la anterior problemática descrita con el uso de la función: find_All().

blocks = layout.find('a')
print((blocks)) 




<a href="/wiki/25_de_julio" title="25 de julio">25 de julio</a>


In [360]:
#Obtención de lo contenido en la etiqueta.

blocks.getText()

'25 de julio'

### Extracción de todas las tablas mediando la libería Pandas y su función: read_html. De esa forma, se logró la extracción del contenido de todas las tablas, evitando la problemática de no poseer un id que las diferencie entre si. 

In [365]:
layout2 = soup.find_all(class_ = 'wikitable')

In [366]:
df = pd.read_html(str(layout2))

In [367]:
print(df)

[    Período                                         Período.1  Efigie  \
0       NaN            25 de julio de 1810-1 de abril de 1811     NaN   
1       NaN       1 de abril de 1811-19 de septiembre de 1811     NaN   
2       NaN     30 de septiembre de 1813-29 de agosto de 1815     NaN   
3       NaN  23 de diciembre de 1811 -24 de diciembre de 1811     NaN   
4       NaN           25 de junio de 1812-5 de agosto de 1812     NaN   
5       NaN    6 de noviembre de 1812-14 de diciembre de 1812     NaN   
6       NaN        27 de octubre de 1812-5 de octubre de 1814     NaN   
7       NaN      29 de agosto de 1813-12 de diciembre de 1814     NaN   
8       NaN          5 de octubre de 1814-21 de enero de 1815     NaN   
9       NaN          5 de octubre de 1814-21 de enero de 1815     NaN   
10      NaN          5 de octubre de 1814-21 de enero de 1815     NaN   
11      NaN       28 de noviembre de 1814-28 de marzo de 1815     NaN   
12      NaN           28 de marzo de 1815-28 de ju